In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import re

In [4]:
anime = pd.read_csv('anime.csv')

In [5]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
anime[anime["episodes"]=="Unknown"].head()

,anime_id,name,genre,type,episodes,rating,members
74,21,One Piece,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,Unknown,8.58,504862
252,235,Detective Conan,"Adventure, Comedy, Mystery, Police, Shounen",TV,Unknown,8.25,114702
615,1735,Naruto: Shippuuden,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,Unknown,7.94,533578
991,966,Crayon Shin-chan,"Comedy, Ecchi, Kids, School, Shounen, Slice of...",TV,Unknown,7.73,26267
1021,33157,Tanaka-kun wa Itsumo Kedaruge Specials,"Comedy, School, Slice of Life",Special,Unknown,7.72,5400


In [8]:
anime.loc[(anime["genre"]=="Hentai") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime.loc[(anime["genre"]=="OVA") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime.loc[(anime["genre"]=="Movie") & (anime["episodes"]=="Unknown")] = "1"


In [9]:
known_animes = {"Naruto Shippuuden":500, "One Piece":873,"Detective Conan":932, "Dragon Ball Super":131,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25}

In [10]:
for k,v in known_animes.items():
    anime.loc[anime["name"]==k,"episodes"] = v

In [11]:
anime["episodes"] = anime["episodes"].map(lambda x:np.nan if x=="Unknown" else x)

In [12]:
anime["episodes"].fillna(anime["episodes"].median(),inplace=True)

In [15]:
pd.get_dummies(anime[["type"]]).head()

,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,1,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [16]:
anime["rating"] = anime["rating"].astype(float)
anime["rating"].fillna(anime["rating"].median(),inplace=True)
anime["members"] = anime["members"].astype(float)

In [17]:
anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),
                           pd.get_dummies(anime[["type"]]),
                           anime[["rating"]],anime[["members"]],anime["episodes"]],axis=1)

anime["name"] = anime["name"].map(lambda name:re.sub('[^A-Za-z0-9]+'," ",name))
anime_features.head()

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,Yaoi,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,rating,members,episodes
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,9.37,200630.0,1
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,9.26,793665.0,64
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.25,114262.0,51
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.17,673572.0,24
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.16,151266.0,51


In [18]:
anime_features.columns

Index([u' Adventure', u' Cars', u' Comedy', u' Dementia', u' Demons',
       u' Drama', u' Ecchi', u' Fantasy', u' Game', u' Harem', u' Hentai',
       u' Historical', u' Horror', u' Josei', u' Kids', u' Magic',
       u' Martial Arts', u' Mecha', u' Military', u' Music', u' Mystery',
       u' Parody', u' Police', u' Psychological', u' Romance', u' Samurai',
       u' School', u' Sci-Fi', u' Seinen', u' Shoujo', u' Shoujo Ai',
       u' Shounen', u' Shounen Ai', u' Slice of Life', u' Space', u' Sports',
       u' Super Power', u' Supernatural', u' Thriller', u' Vampire', u' Yaoi',
       u' Yuri', u'Action', u'Adventure', u'Cars', u'Comedy', u'Dementia',
       u'Demons', u'Drama', u'Ecchi', u'Fantasy', u'Game', u'Harem', u'Hentai',
       u'Historical', u'Horror', u'Josei', u'Kids', u'Magic', u'Martial Arts',
       u'Mecha', u'Military', u'Music', u'Mystery', u'Parody', u'Police',
       u'Psychological', u'Romance', u'Samurai', u'School', u'Sci-Fi',
       u'Seinen', u'Shoujo', u'S

In [20]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
anime_features = min_max_scaler.fit_transform(anime_features)

In [24]:
np.round(anime_features,2)

array([[0.  , 0.  , 0.  , ..., 0.92, 0.2 , 0.  ],
       [1.  , 0.  , 0.  , ..., 0.91, 0.78, 0.03],
       [0.  , 0.  , 1.  , ..., 0.91, 0.11, 0.03],
       ...,
       [0.  , 0.  , 0.  , ..., 0.39, 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.4 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.45, 0.  , 0.  ]])

In [25]:
from sklearn.neighbors import NearestNeighbors

In [26]:
nbrs = NearestNeighbors(n_neighbors = 6,algorithm="ball_tree").fit(anime_features)
distances, indices = nbrs.kneighbors(anime_features)

In [27]:
def get_index_from_name(name):
    return anime[anime["name"]==name].index.tolist()[0]

In [28]:
all_anime_names = list(anime.name.values)

In [29]:
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [30]:
def print_similar_anime(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.ix[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(anime.ix[id]["name"])

In [32]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [34]:
print_similar_anime("Naruto")

Naruto Shippuuden
Katekyo Hitman Reborn 
Bleach
Dragon Ball Z
Boku no Hero Academia


In [35]:
print_similar_anime("Bleach")

Katekyo Hitman Reborn 
Code Breaker
Tokyo ESP
Yozakura Quartet Hana no Uta
Yozakura Quartet


In [37]:
print_similar_anime("Dragon Ball Z")

Dragon Ball Kai
Dragon Ball Super
Dragon Ball Kai 2014 
Naruto Shippuuden
Naruto


In [38]:
get_id_from_partial_name("Naruto")

('Boruto Naruto the Movie', 486)
('Naruto Shippuuden', 615)
('The Last Naruto the Movie', 719)
('Naruto Shippuuden Movie 6 Road to Ninja', 784)
('Naruto', 841)
('Boruto Naruto the Movie Naruto ga Hokage ni Natta Hi', 1103)
('Naruto Shippuuden Movie 5 Blood Prison', 1237)
('Naruto x UT', 1343)
('Naruto Shippuuden Movie 4 The Lost Tower', 1472)
('Naruto Shippuuden Movie 3 Hi no Ishi wo Tsugu Mono', 1573)
('Naruto Shippuuden Movie 1', 1827)
('Naruto Shippuuden Movie 2 Kizuna', 1828)
('Naruto Shippuuden Shippuu quot Konoha Gakuen quot Den', 2374)
('Naruto Honoo no Chuunin Shiken Naruto vs Konohamaru ', 2416)
('Naruto SD Rock Lee no Seishun Full Power Ninden', 2457)
('Naruto Shippuuden Sunny Side Battle', 2458)
('Naruto Movie 1 Dai Katsugeki Yuki Hime Shinobu Houjou Dattebayo ', 2756)
('Naruto Soyokazeden Movie Naruto to Mashin to Mitsu no Onegai Dattebayo ', 2997)
('Naruto Movie 2 Dai Gekitotsu Maboroshi no Chiteiiseki Dattebayo ', 3449)
('Naruto Dai Katsugeki Yuki Hime Shinobu Houjou Datt